# 영어 Word2Vec 만들기

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [6]:
#urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")


('ted_en-20160408.xml', <http.client.HTTPMessage at 0x224340d4b80>)

In [11]:
targetXML=open('F:/Workspace/data/ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [12]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [13]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## Word2Vec 훈련시키기

In [25]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [26]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.84520024061203), ('guy', 0.8256785869598389), ('girl', 0.7708662152290344), ('lady', 0.7635362148284912), ('boy', 0.7575492262840271), ('gentleman', 0.7419318556785583), ('soldier', 0.725075364112854), ('kid', 0.7011024355888367), ('poet', 0.6880499720573425), ('son', 0.665919840335846)]


In [27]:
model.wv.most_similar(positive=['man','girl'], negative=['boy'], topn=3)

[('woman', 0.8385322093963623),
 ('lady', 0.7525697946548462),
 ('guy', 0.743120551109314)]

In [29]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('F:/Workspace/data/eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("F:/Workspace/data/eng_w2v") # 모델 로드

In [30]:
loaded_model.most_similar("man")

[('woman', 0.84520024061203),
 ('guy', 0.8256785869598389),
 ('girl', 0.7708662152290344),
 ('lady', 0.7635362148284912),
 ('boy', 0.7575492262840271),
 ('gentleman', 0.7419318556785583),
 ('soldier', 0.725075364112854),
 ('kid', 0.7011024355888367),
 ('poet', 0.6880499720573425),
 ('son', 0.665919840335846)]